<a href="https://colab.research.google.com/github/trie0000/external/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""DL_Basic_2025_Competition_NYUv2_baseline.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/17t7uAU0aST5aUt6sIJCqFneGlQrxFrJY

# Deep Learning 基礎講座　最終課題: NYUv2 セマンティックセグメンテーション

## 概要
RGB画像から、画像内の各ピクセルがどのクラスに属するかを予測するセマンティックセグメンテーションタスク.

### データセット
- データセット: NYUv2 dataset
- 訓練データ: 795枚
- テストデータ: 654枚
- 入力: RGB画像 + 深度マップ（元画像サイズは可変）
- 出力: 13クラスのセグメンテーションマップ
- 評価指標: Mean IoU (Intersection over Union)

### データセットの詳細（[NYU Depth Dataset V2](https://cs.nyu.edu/~fergus/datasets/nyu_depth_v2.html)）
- 画像は屋内シーンを撮影したもので、家具や壁、床などの物体が含まれています.
- 各画像に対して13クラスのセグメンテーションラベルが提供されます.
- データは以下のディレクトリ構造で提供:
```
data/NYUv2/
├─train/
│  ├─image/      # RGB画像
│  │    000000.png
│  │    ...
│  │
│  ├─depth/      # 深度マップ
│  │    000000.png
│  │    ...
│  │
│  └─label/      # 13クラスセグメンテーション（教師ラベル）
│       000000.png
│       ...
└─test/
   ├─image/      # RGB画像
   │    000000.png
   │    ...
   │  ├─depth/   # 深度マップ
   │    000000.png
   │    ...
```

### タスクの詳細
- 入力のRGB画像と深度マップから、各ピクセルが13クラスのどれに属するかを予測するタスクです.
- 評価はMean IoUを使用します．
  - 各クラスごとにIoUを計算し、その平均を取ります.
  - IoUは以下の式で計算:
  $$IoU = \frac{TP}{TP + FP + FN}$$
    - TP: True Positive（正しく予測されたピクセル数）
    - FP: False Positive（誤って予測されたピクセル数）
    - FN: False Negative（見逃したピクセル数）

### 前処理
- 入力画像は512×512にリサイズされます.
- ピクセル値は0-1に正規化されます.
- セグメンテーションラベルは0-12の整数値（13クラス）です．
  - 255はignore index（評価から除外）

### 提出形式
- テスト画像（RGB + Depth）の各ピクセルに対してクラス（0~12）を予測したものをnumpy配列として保存されます.
- ファイル名: `submission.npy`
- 配列の形状: [テストデータ数, 高さ, 幅]
- 各ピクセルの値: 0-12の整数（予測クラス）

## 考えられる工夫の例
- 事前学習モデルの fine-tuning
    - ImageNetなどで事前学習されたモデルを本データセットでfine-tuningすることで性能向上が見込めます.
- 損失関数の再設計
    - クラスごとの出現頻度に応じて損失を補正するように損失関数を設計すると、クラス分布の不均衡に対してロバストな学習ができます.
- 画像の前処理
    - RandomResizedCrop / Flip / ColorJitter 等のデータ拡張を追加することで，汎化性能の向上が見込めます．

## 修了要件を満たす条件
- ベースラインでは，omnicampus 上での性能評価において， 38.2% となります．したがって，ベースラインである 38.2% を超えた提出のみ，修了要件として認めます．
- ベースラインから改善を加えることで， 50%以上に性能向上することを運営で確認しています．こちらを 1つの指標として取り組んでみてください．

## 注意点
- 最終的な予測モデルは，**配布している訓練データを用いて学習**（ファインチューニング含む）したものとしてください．
- 学習を行わず，**事前学習済みモデルの知識のみを利用した推論は禁止**します．
（例: ChatGPT 等の LLM に入力して推論を得るのみ）

### 事前学習モデルの利用
許可される事項
- **構成要素としての事前学習モデルの利用**: 自身で実装したアーキテクチャの一部（特徴抽出，埋め込みなど）として事前学習モデル（BERT，ViT など）を利用することは可能です．
- **ファインチューニング**: 上記の用途で利用している事前学習モデルのファインチューニングは可能です．

禁止される事項
- **タスク解決用の事前学習モデルの利用**: transformers などで提供されている，対象タスクを直接解くための事前学習モデルでそのまま推論のみ，またはファインチューニングのみで利用することは禁止とします．
  - 禁止事項の例: VQA タスクを直接解くための事前学習モデルを VQA タスクで利用する．

### データの準備
データをダウンロードした際に，google drive したため，利用するために google drive をマウントする必要があります．また， drive 上で展開することができないため，/content ディレクトリ下にコピーし "data.zip" を展開します．
google drive 上に "data.zip" が配置されていない場合は実行できません．google drive 上に "data.zip" (**831MB**) を配置することが可能であれば，"data_download.ipynb" を先に実行してください．難しい場合は，omnicampus 演習環境を利用してください．．
"""

# omnicampus 上では 4 セル目まで実行不要
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# データダウンロード用の notebook にてgoogle drive への保存後，
# 反映に時間がかかる可能性がありますので，google drive のマウント後，
# data.zip がディレクトリ内にあることを確認してから実行してください．
# data.zip を /content 下にコピーする
!cp "/content/drive/MyDrive/data.zip" "/content"

# Commented out IPython magic to ensure Python compatibility.
# カレントディレクトリ下のファイル群を確認
# data.zip が表示されれば問題ないです
# %ls

# データを解凍する
!unzip data.zip
!mkdir data
!mv train test data/

"""omnicampus 演習環境では，data_download.ipynb のマウント，zip 化，drive へのコピーを実行しないことで，"data.zip" を解凍した形で配置されます．したがって，data ディレクトリが存在するディレクトリをカレントディレクトリとするだけで良いです．


"""

# Commented out IPython magic to ensure Python compatibility.
# omnicampus 実行用
# 以下の例では/workspace/Segmentation/split_data_scripts/omnicampus に data ディレクトリがあると想定
# %cd /workspace/Segmentation/split_data_scripts_omnicampus

# omnicampus 実行用
!pip install h5py scikit-image

"""# import library"""

Mounted at /content/drive
Archive:  data.zip
  inflating: data/train/image/000600.png  
  inflating: data/train/image/000320.png  
  inflating: data/train/image/000491.png  
  inflating: data/train/image/000502.png  
  inflating: data/train/image/000129.png  
  inflating: data/train/image/000044.png  
  inflating: data/train/image/000652.png  
  inflating: data/train/image/000919.png  
  inflating: data/train/image/000528.png  
  inflating: data/train/image/000853.png  
  inflating: data/train/image/000177.png  
  inflating: data/train/image/000584.png  
  inflating: data/train/image/001319.png  
  inflating: data/train/image/000597.png  
  inflating: data/train/image/000223.png  
  inflating: data/train/image/001350.png  
  inflating: data/train/image/000404.png  
  inflating: data/train/image/000488.png  
  inflating: data/train/image/000268.png  
  inflating: data/train/image/000481.png  
  inflating: data/train/image/000341.png  
  inflating: data/train/image/000159.png  
  inflati

'# import library'

In [15]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import os
import time
from tqdm import tqdm
import numpy as np
from scipy.io import loadmat
from PIL import Image
import torch
import torch.nn as nn
from torch import optim
import torch.utils.data as data
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import VisionDataset
from torchvision import transforms
from torchvision.transforms import (
    Compose,
    RandomResizedCrop,
    RandomHorizontalFlip,
    ColorJitter,
    GaussianBlur,
    Resize,
    ToTensor,
    Normalize,
    Lambda,
    InterpolationMode
)
from torch.cuda.amp import autocast, GradScaler
from dataclasses import dataclass
import random

from torchvision.models.segmentation import deeplabv3_resnet50


"""# DataLoader"""

# カラーマップ生成関数：セグメンテーションの可視化用
def colormap(N=256, normalized=False):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap/255 if normalized else cmap
    return cmap

# NYUv2データセット：RGB画像、セグメンテーション、深度、法線マップを提供するデータセット
class NYUv2(VisionDataset):
    """NYUv2 dataset

    Args:
        root (string): Root directory path.
        split (string, optional): 'train' for training set, and 'test' for test set. Default: 'train'.
        target_type (string, optional): Type of target to use, ``semantic``, ``depth``.
        transform (callable, optional): A function/transform that takes in an PIL image and returns a transformed version.
        target_transform (callable, optional): A function/transform that takes in the target and transforms it.
    """
    cmap = colormap()
    def __init__(self,
                 root,
                 split='train',
                 include_depth=False,
                 transform=None,
                 target_transform=None,
                 ):
        super(NYUv2, self).__init__(root, transform=transform, target_transform=target_transform)

        # データセットの基本設定
        assert(split in ('train', 'test'))
        self.root = root
        self.split = split
        self.include_depth = include_depth
        self.train_idx = np.array([255, ] + list(range(13)))  # 13クラス分類用

        # 画像ファイルのパスリストを作成
        img_names = os.listdir(os.path.join(self.root, self.split, 'image'))
        img_names.sort()
        images_dir = os.path.join(self.root, self.split, 'image')
        self.images = [os.path.join(images_dir, name) for name in img_names]

        label_dir = os.path.join(self.root, self.split, 'label')
        if (self.split == 'train'):
          self.labels = [os.path.join(label_dir, name) for name in img_names]
          self.targets = self.labels

        depth_dir = os.path.join(self.root, self.split, 'depth')
        self.depths = [os.path.join(depth_dir, name) for name in img_names]

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert("RGB")
        depth = Image.open(self.depths[idx]).convert("L")  # 深度を必ず1chにする

        if self.transform is not None:
            image = self.transform(image)
            depth = self.transform(depth)
        if self.split=='test':
          if self.include_depth:
              return image, depth
          return image
        if self.split == 'train' and self.target_transform is not None:
            target = Image.open(self.targets[idx])
            target = self.target_transform(target)
        if self.include_depth:
              return image, depth, target

        return image, target

    def __len__(self):
        return len(self.images)

"""# Model Section"""

# DeepLabV3-ResNet50 の最初のconvを 4ch に変更（RGB+Depth）
def patch_first_conv_to_4ch(model):
    if hasattr(model.backbone, "conv1"):
        conv1 = model.backbone.conv1
        body = model.backbone
    elif hasattr(model.backbone, "body") and hasattr(model.backbone.body, "conv1"):
        conv1 = model.backbone.body.conv1
        body = model.backbone.body
    else:
        raise RuntimeError("conv1 が見つかりません。")

    new_conv1 = nn.Conv2d(
        in_channels=4,
        out_channels=conv1.out_channels,
        kernel_size=conv1.kernel_size,
        stride=conv1.stride,
        padding=conv1.padding,
        bias=(conv1.bias is not None),
    ).to(conv1.weight.device)  # conv1と同じdeviceへ

    with torch.no_grad():
        new_conv1.weight[:, :3] = conv1.weight
        new_conv1.weight[:, 3:4] = conv1.weight.mean(dim=1, keepdim=True)

    body.conv1 = new_conv1

"""# Train and Valid"""

# config
@dataclass
class TrainingConfig:
    # データセットパス
    dataset_root: str = "data"

    # データ関連
    batch_size: int = 32
    num_workers: int = 4

    # モデル関連
    in_channels: int = 3
    num_classes: int = 13  # NYUv2データセットの場合

    # 学習関連
    epochs: int = 100
    learning_rate: float = 0.001
    weight_decay: float = 1e-4

    # データ分割関連
    train_val_split: float = 0.8  # 訓練データの割合

    # デバイス設定
    device: str = "cuda" if torch.cuda.is_available() else "cpu"

    # チェックポイント関連
    checkpoint_dir: str = "checkpoints"
    save_interval: int = 5  # エポックごとのモデル保存間隔

    # データ拡張・前処理関連
    image_size: tuple = (256, 256)
    normalize_mean: tuple = (0.485, 0.456, 0.406)  # ImageNetの標準化パラメータ
    normalize_std: tuple = (0.229, 0.224, 0.225)

    def __post_init__(self):
        import os
        os.makedirs(self.checkpoint_dir, exist_ok=True)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# 設定の初期化
config = TrainingConfig(
    dataset_root='/content/data',
    batch_size=16,
    num_workers=4,
    learning_rate=1e-4,
    epochs=50,
#    epochs=10,
    image_size=(320, 240),
    in_channels=4  # RGB(3チャネル) + Depth(1チャネル)
)

# ------------------
#    Dataloader
# ------------------

transform = Compose([
    Resize(config.image_size, interpolation=InterpolationMode.BILINEAR),
    ToTensor()
])

target_transform = Compose([
    Resize(config.image_size, interpolation=InterpolationMode.NEAREST),
    Lambda(lambda lbl: torch.from_numpy(np.array(lbl)).long())
])

train_dataset = NYUv2(
    root=config.dataset_root,
    split='train',
    include_depth=True,
    transform=transform,
    target_transform=target_transform
)

test_dataset = NYUv2(
    root=config.dataset_root,
    split='test',
    include_depth=True,
    transform=transform
)

train_data = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, num_workers=config.num_workers)
test_data = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=config.num_workers)

device = config.device
print(f"Using device: {device}")

# ------------------
#    Model
# ------------------
model = deeplabv3_resnet50(weights=None, num_classes=config.num_classes)  # まずCPU上で作る
patch_first_conv_to_4ch(model)                                            # CPU上でconv差し替え
model = model.to(device)                                                  # 最後にGPUへ移動

# ------------------
#    optimizer
# ------------------
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)
criterion = nn.CrossEntropyLoss(ignore_index=255)

# ------------------
#    Training
# ------------------
num_epochs = config.epochs
scaler = GradScaler()

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    print(f"on epoch: {epoch+1}")
    with tqdm(train_data) as pbar:
        for batch_idx, (image, depth, label) in enumerate(pbar):
            image, depth, label = image.to(device), depth.to(device), label.to(device)
            optimizer.zero_grad()

            with autocast():
                x = torch.cat((image, depth), dim=1)  # RGB + Depth
                pred = model(x)["out"]                # DeepLabは dict
                loss = criterion(pred, label)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            total_loss += loss.item()
            del image, depth, label, pred, loss

    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_data)}')

# モデルの保存
current_time = time.strftime("%Y%m%d%H%M%S")
model_path = f"model_{current_time}.pt"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# ------------------
#    Evaluation
# ------------------

model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

predictions = []

with torch.no_grad():
    print("Generating predictions...")
    for image, depth in tqdm(test_data):
        image, depth = image.to(device), depth.to(device)
        x = torch.cat((image, depth), dim=1)
        output = model(x)["out"]       # DeepLabは dict
        pred = output.argmax(dim=1)    # [Batch, H, W]
        predictions.append(pred.cpu())

predictions = torch.cat(predictions, dim=0)
predictions = predictions.cpu().numpy()
np.save('submission.npy', predictions)
print("Predictions saved to submission.npy")

"""## 提出方法

以下の3点をzip化し，Omnicampusの「最終課題 (セグメンテーション)」から提出してください．

- `submission.npy`
- `model.pt`や`model_best.pt`など，テストに使用した重み（拡張子は`.pt`のみ）
- 本Colab Notebook
"""

from zipfile import ZipFile, ZIP_DEFLATED

notebook_path = "/content/drive/MyDrive/code.ipynb"

with ZipFile("submission.zip",
             mode="w",
             compression=ZIP_DEFLATED,
             compresslevel=9) as zf:
    zf.write("submission.npy")
    zf.write(model_path)
#    zf.write(notebook_path,
#             arcname="code.ipynb")


Using device: cuda


/tmp/ipython-input-2787394586.py:267: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


on epoch: 1


  0%|          | 0/50 [00:00<?, ?it/s]/tmp/ipython-input-2787394586.py:278: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 50/50 [00:06<00:00,  7.25it/s]


Epoch 1, Loss: 1.5488601160049438
on epoch: 2


...

Epoch 49, Loss: 0.04250767961144447
on epoch: 50


100%|██████████| 50/50 [00:06<00:00,  7.40it/s]


Epoch 50, Loss: 0.042516489923000333
Model saved to model_20260102095840.pt
Generating predictions...


100%|██████████| 654/654 [00:11<00:00, 54.59it/s]


Predictions saved to submission.npy


In [ ]:
# ------------------
#    Evaluation (DeepLabV3-ResNet50 / ResNet backbone)
# ------------------

ckpt = torch.load(final_path, map_location=device)

# もし「学習で checkpoint dict（model_state入り）」で保存している場合にも対応
if isinstance(ckpt, dict) and "model_state" in ckpt:
    model.load_state_dict(ckpt["model_state"])
else:
    model.load_state_dict(ckpt)

model.eval()

predictions = []

with torch.no_grad():
    print("Generating predictions...")
    for image, depth in tqdm(test_data):
        image = image.to(device, non_blocking=True)
        depth = depth.to(device, non_blocking=True)

        # 念のため：depthが3ch事故のときは1chに落とす
        if depth.dim() == 4 and depth.size(1) == 3:
            depth = depth[:, :1, :, :]

        x = torch.cat((image, depth), dim=1)   # [B,4,H,W]

        # DeepLabV3 は dict を返すので "out" を使う
        out = model(x)["out"]                  # [B,num_classes,H,W]
        pred = out.argmax(dim=1)               # [B,H,W]

        predictions.append(pred.cpu())

predictions = torch.cat(predictions, dim=0).numpy()
np.save("submission.npy", predictions)
print("Predictions saved to submission.npy")


"""## 提出方法

以下の3点をzip化し，Omnicampusの「最終課題 (セグメンテーション)」から提出してください．

- `submission.npy`
- `model.pt`や`model_best.pt`など，テストに使用した重み（拡張子は`.pt`のみ）
- 本Colab Notebook
"""

from zipfile import ZipFile, ZIP_DEFLATED

notebook_path = "/content/drive/MyDrive/Colab Notebooks/DL_Basic_2025_Competition_NYUv2_baseline.ipynb"

from zipfile import ZipFile, ZIP_DEFLATED

notebook_path = "/content/drive/MyDrive/code.ipynb"

with ZipFile("submission.zip",
             mode="w",
             compression=ZIP_DEFLATED,
             compresslevel=9) as zf:
    zf.write("submission.npy")
    zf.write(model_path)
#    zf.write(notebook_path,
#             arcname="code.ipynb")


Generating predictions...


100%|██████████| 654/654 [00:12<00:00, 53.10it/s]


Predictions saved to submission.npy
